# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
    "model": MODEL,
    "messages": messages,
    "stream": False
}

In [ ]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

/bin/bash: line 1: ollama: command not found


In [5]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can generate high-quality content such as articles, blog posts, social media posts, and even entire books. This can help businesses save time and money on content creation, while also improving the consistency and quality of their content.
2. **Marketing Automation**: Generative AI can be used to create personalized marketing messages, product recommendations, and customer personas. This can help businesses personalize their marketing efforts and improve their return on investment (ROI).
3. **Customer Service Chatbots**: Generative AI-powered chatbots can understand natural language and respond to customer inquiries in a more human-like way. This can help businesses provide 24/7 customer support and improve the overall customer experience.
4. **Product Design and Development**: Generative AI can be used to design and develop new products, such as f

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [6]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can be used to generate high-quality content such as articles, blog posts, social media posts, and even entire books. This can help businesses save time and resources while maintaining consistency in their content.
2. **Product Design**: Generative AI can be used to design new products, such as furniture, jewelry, or electronics. It can also be used to generate 3D models and prototypes for product development.
3. **Marketing Automation**: Generative AI can be used to create personalized marketing messages, emails, and social media posts that are tailored to individual customers' interests and preferences.
4. **Customer Service Chatbots**: Generative AI can be used to power chatbots that provide 24/7 customer support and help resolve simple queries quickly.
5. **Personalized Recommendations**: Generative AI can be used to generate personalized product recommenda

## Alternative approach - using OpenAI python library to connect to Ollama

In [7]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI (GA) has numerous exciting business applications across various industries. Here are some examples:

1. **Content Creation**: GA algorithms can generate high-quality content, such as news articles, social media posts, product descriptions, and even entire books. Companies like WordLift and Content Blossom use GA to produce engaging content for websites and blogs.
2. **Personalized Product Recommendations**: Generative AI tools like Google Lens and Amazon's SageMaker can analyze products' features and generate personalized product recommendations based on user behavior and preferences.
3. **Chatbots and Virtual Assistants**: GA enables the creation of highly realistic chatbots and virtual assistants that can mimic human-like conversations, improving customer experience and support for businesses like Microsoft, IBM, and Google.
4. **Marketing and Advertising**: Generative AI algorithms can generate personalized ads, product descriptions, and even entire campaigns using dat

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [8]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [9]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [10]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = (
    "You are an assistant that analyzes the contents of a website"
    "and provides a short summary, ignoring text that might be navigation related."
    "Respond in markdown."
)

In [11]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += (
        "\nThe contents of this website is as follows;"
        "please provide a short summary of this website in markdown."
        "If it includes news or announcements, then summarize these too.\n\n"
    )
    user_prompt += website.text
    return user_prompt

In [12]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=messages_for(website)
    )
    return response.choices[0].message.content

In [14]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [15]:
display_summary("https://edwarddonner.com")

**Summary**
===============

*   The website is maintained by Edward Donner, a co-founder and CTO of Nebula.io.
*   Nebula.io applies AI to help people discover their potential and pursue their reason for being.
*   They develop proprietary LLMs verticalized in the field of talent acquisition and management.

**News/Announcements**
=============================

*   **Welcome, SuperDataScientists! (December 21, 2024)**: A message from Edward to his audience, possibly announcing a new initiative or community.
*   **Mastering AI and LLM Engineering – Resources (October 16, 2024)**: Edward shares resources that can help professionals master AI and LLM engineering.
*   **From Software Engineer to AI Data Scientist – resources (August 6, 2024)**: Another resource sharing post by Edward on transitioning into an AI Data Scientist role.
*   **Outsmart LLM Arena – a battle of diplomacy and deviousness**: An arena where LLMs compete in diplomacy and cunning challenges. This seems to be more of a showcase or entertainment feature rather than a news announcement.

Note: There are no recent news announcements, but a welcome message targeting the "SuperDataScientists" audience.